# Trace with Python SDK - Tutorial

This tutorial demonstrates how to use the Agenta Python SDK to trace your LLM applications. You'll learn how to:

- Set up tracing with the Agenta SDK
- Instrument functions and OpenAI calls automatically
- Start and end spans manually to capture internals
- Reference prompt versions in your traces
- Redact sensitive data from traces

## What You'll Build

We'll create a simple LLM application that:
1. Uses OpenAI auto-instrumentation to trace API calls
2. Instruments custom functions to capture workflow steps
3. Stores internal data like retrieved context
4. Links traces to deployed prompt versions
5. Redacts sensitive information from traces

## Install Dependencies

In [ ]:
pip install -U agenta openai opentelemetry-instrumentation-openai

## Setup

Before using the SDK, we need to initialize it with your API keys. The SDK requires:
- **Agenta API Key**: For sending traces to Agenta
- **OpenAI API Key**: For making LLM calls

You can get your Agenta API key from the [API Keys page](https://cloud.agenta.ai/settings?tab=apiKeys).

In [ ]:
import os

os.environ["AGENTA_HOST"] = (
    "https://cloud.agenta.ai/"  # Default value, change for self-hosted
)
os.environ["AGENTA_API_KEY"] = ""
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
import os
import agenta as ag
from getpass import getpass

# Initialize the SDK with your API key
api_key = os.getenv("AGENTA_API_KEY")
if not api_key:
    os.environ["AGENTA_API_KEY"] = getpass("Enter your Agenta API key: ")

openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

# Initialize the SDK
ag.init()

## Part 1: Setup Tracing with OpenAI Auto-Instrumentation

The Agenta SDK provides two powerful mechanisms for tracing:

1. **Auto-instrumentation**: Automatically traces third-party libraries like OpenAI
2. **Function decorators**: Manually instrument your custom functions

Let's start by setting up OpenAI auto-instrumentation, which will capture all OpenAI API calls automatically.

In [ ]:
from opentelemetry.instrumentation.openai import OpenAIInstrumentor
import openai

# Instrument OpenAI to automatically trace all API calls
OpenAIInstrumentor().instrument()

print("OpenAI auto-instrumentation enabled!")

## Part 2: Instrument Functions

Now let's create a simple function and instrument it using the `@ag.instrument()` decorator. This will create a span for the function and automatically capture its inputs and outputs.

The decorator accepts a `spankind` parameter to categorize the span. Available types include: `agent`, `chain`, `workflow`, `tool`, `embedding`, `query`, `completion`, `chat`, `rerank`.

In [ ]:
@ag.instrument(spankind="workflow")
def generate_story(topic: str):
    """Generate a short story about the given topic."""
    response = openai.chat.completions.create(
        model="gpt-5",
        messages=[
            {"role": "system", "content": "You are a creative storyteller."},
            {"role": "user", "content": f"Write a short story about {topic}."},
        ],
    )
    return response.choices[0].message.content


# Test the instrumented function
story = generate_story("AI Engineering")
print("Generated story:")
print(story)

## Part 3: Starting Spans and Storing Internals

Sometimes you need to capture intermediate data that isn't part of the function's inputs or outputs. The SDK provides two methods:

- `ag.tracing.store_meta()`: Add metadata to a span (saved under `ag.meta`)
- `ag.tracing.store_internals()`: Store internal data (saved under `ag.data.internals`)

Internals are especially useful because they:
1. Are searchable using plain text queries
2. Appear in the overview tab of the observability drawer

Let's create a RAG (Retrieval-Augmented Generation) example that captures the retrieved context:

In [ ]:
@ag.instrument(spankind="tool")
def retrieve_context(query: str):
    """Simulate retrieving context from a knowledge base."""
    # In a real application, this would query a vector database
    context = [
        "Agenta is an open-source LLM developer platform.",
        "Agenta provides tools for prompt management, evaluation, and observability.",
        "The Agenta SDK supports tracing with OpenTelemetry.",
    ]

    # Store metadata about the retrieval
    ag.tracing.store_meta(
        {"retrieval_method": "vector_search", "num_results": len(context)}
    )

    return context


@ag.instrument(spankind="workflow")
def rag_workflow(query: str):
    """Answer a question using retrieved context."""
    # Retrieve context
    context = retrieve_context(query)

    # Store the retrieved context as internals
    # This makes it visible in the UI and searchable
    ag.tracing.store_internals({"retrieved_context": context})

    # Generate answer using context
    context_str = "\n".join(context)
    prompt = f"Answer the following question based on the context:\n\nContext:\n{context_str}\n\nQuestion: {query}"

    response = openai.chat.completions.create(
        model="gpt-5",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant. Answer questions based only on the provided context.",
            },
            {"role": "user", "content": prompt},
        ],
    )

    return response.choices[0].message.content


# Test the RAG workflow
answer = rag_workflow("What is Agenta?")
print("Answer:")
print(answer)

## Part 4: Reference Prompt Versions

One of Agenta's powerful features is linking traces to specific prompt versions. This allows you to:
- Filter traces by application, variant, or environment
- Compare performance across different variants
- Track production behavior

Let's create a prompt using the SDK, deploy it, and then reference it in our traces.

### Create and Deploy a Prompt

In [ ]:
from agenta.sdk.types import PromptTemplate, Message, ModelConfig
from pydantic import BaseModel


# Define the prompt configuration
class Config(BaseModel):
    prompt: PromptTemplate


config = Config(
    prompt=PromptTemplate(
        messages=[
            Message(
                role="system",
                content="You are a helpful assistant that explains topics clearly.",
            ),
            Message(role="user", content="Explain {{topic}} in simple terms."),
        ],
        llm_config=ModelConfig(
            model="gpt-5",
            max_tokens=200,
            temperature=0.7,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0,
        ),
        template_format="curly",
    )
)

# Create an application and variant
app = ag.AppManager.create(
    app_slug="topic-explainer-traced",
    template_key="SERVICE:completion",
)

print(f"Created application: {app.app_name}")

# Create a variant with the prompt configuration
variant = ag.VariantManager.create(
    parameters=config.model_dump(),
    app_slug="topic-explainer-traced",
    variant_slug="production-variant",
)

print(f"Created variant: {variant.variant_slug} (version {variant.variant_version})")

# Deploy to production environment
deployment = ag.DeploymentManager.deploy(
    app_slug="topic-explainer-traced",
    variant_slug="production-variant",
    environment_slug="production",
)

print(f"Deployed to {deployment.environment_slug} environment")

### Reference the Prompt in Traces

Now we'll create a function that uses the deployed prompt and links its traces to the application and environment.

In [ ]:
@ag.instrument(spankind="workflow")
def explain_topic_with_prompt(topic: str):
    """Explain a topic using the deployed prompt configuration."""

    # Fetch the prompt configuration from production
    prompt_config = ag.ConfigManager.get_from_registry(
        app_slug="topic-explainer-traced", environment_slug="production"
    )

    # Format the prompt with the topic
    prompt_template = PromptTemplate(**prompt_config["prompt"])
    formatted_prompt = prompt_template.format(topic=topic)

    # Make the OpenAI call
    response = openai.chat.completions.create(**formatted_prompt.to_openai_kwargs())

    # Link this trace to the application and environment
    ag.tracing.store_refs(
        {
            "application.slug": "topic-explainer-traced",
            "variant.slug": "production-variant",
            "environment.slug": "production",
        }
    )

    return response.choices[0].message.content


# Test the function
explanation = explain_topic_with_prompt("machine learning")
print("Explanation:")
print(explanation)

## Part 5: Redact Sensitive Data

When working with production data, you often need to exclude sensitive information from traces. The Agenta SDK provides several ways to redact data:

1. **Simple redaction**: Ignore all inputs/outputs
2. **Selective redaction**: Ignore specific fields
3. **Custom redaction**: Use a callback function for fine-grained control
4. **Global redaction**: Apply rules across all instrumented functions

Let's explore these different approaches.

### Simple Redaction: Ignore All Inputs/Outputs

In [ ]:
@ag.instrument(spankind="workflow", ignore_inputs=True, ignore_outputs=True)
def process_sensitive_data(user_email: str, credit_card: str):
    """Process sensitive data without logging inputs/outputs."""
    # The function inputs and outputs won't be captured in the trace
    result = f"Processed data for {user_email}"
    return result


# This trace will not contain inputs or outputs
result = process_sensitive_data("user@example.com", "4111-1111-1111-1111")
print(result)

### Selective Redaction: Ignore Specific Fields

In [ ]:
@ag.instrument(
    spankind="workflow",
    ignore_inputs=["api_key", "password"],
    ignore_outputs=["internal_token"],
)
def authenticate_user(username: str, password: str, api_key: str):
    """Authenticate a user (password and api_key will be redacted)."""
    # Simulate authentication
    return {
        "username": username,
        "authenticated": True,
        "internal_token": "secret-token-12345",  # This will be redacted
    }


# The trace will show username but not password or api_key
auth_result = authenticate_user("john_doe", "secret123", "sk-abc123")
print(f"Authenticated: {auth_result['authenticated']}")

### Custom Redaction: Use a Callback Function

In [ ]:
import re


def redact_pii(name: str, field: str, data: dict):
    """Custom redaction function that removes PII."""
    if field == "inputs":
        # Redact email addresses
        if "email" in data:
            data["email"] = "[REDACTED]"
        # Redact phone numbers
        if "phone" in data:
            data["phone"] = "[REDACTED]"

    if field == "outputs":
        # Redact any credit card patterns
        if isinstance(data, dict):
            for key, value in data.items():
                if isinstance(value, str):
                    # Simple credit card pattern
                    data[key] = re.sub(
                        r"\d{4}[-\s]?\d{4}[-\s]?\d{4}[-\s]?\d{4}",
                        "[CARD-REDACTED]",
                        value,
                    )

    return data


@ag.instrument(
    spankind="workflow",
    redact=redact_pii,
    redact_on_error=False,  # Don't apply redaction if it raises an error
)
def process_customer_order(name: str, email: str, phone: str, card_number: str):
    """Process a customer order with PII redaction."""
    return {
        "status": "processed",
        "customer": name,
        "payment_info": f"Charged card ending in {card_number[-4:]}",
        "full_card": card_number,  # This will be redacted
    }


# Test with sample data
order = process_customer_order(
    name="Jane Smith",
    email="jane@example.com",  # Will be redacted
    phone="555-1234",  # Will be redacted
    card_number="4111-1111-1111-1111",  # Will be redacted in output
)
print(f"Order status: {order['status']}")

### Global Redaction: Apply Rules Across All Functions

For organization-wide policies, you can set up global redaction rules during initialization. Note: Since we already called `ag.init()`, this is just for demonstration. In a real application, you would set this during the initial setup.

In [ ]:
# Example of global redaction setup (would be done during ag.init())
from typing import Dict, Any


def global_redact_function(name: str, field: str, data: Dict[str, Any]):
    """Global redaction that applies to all instrumented functions."""
    # Remove any field containing 'api_key' or 'secret'
    if isinstance(data, dict):
        keys_to_redact = [
            k for k in data.keys() if "api_key" in k.lower() or "secret" in k.lower()
        ]
        for key in keys_to_redact:
            data[key] = "[REDACTED]"

    return data


# In production, you would initialize like this:
# ag.init(
#     redact=global_redact_function,
#     redact_on_error=True
# )

print("Global redaction would be configured during ag.init()")

## Summary

In this tutorial, you learned how to:

1. ✅ **Set up tracing** with the Agenta SDK and OpenAI auto-instrumentation
2. ✅ **Instrument functions** using the `@ag.instrument()` decorator
3. ✅ **Store internals and metadata** to capture intermediate data in your workflows
4. ✅ **Reference prompt versions** by creating, deploying, and linking traces to applications
5. ✅ **Redact sensitive data** using multiple approaches for privacy protection

## Next Steps

- Explore [distributed tracing](/observability/trace-with-python-sdk/distributed-tracing) for multi-service applications
- Learn about [cost tracking](/observability/trace-with-python-sdk/track-costs) to monitor LLM expenses
- Understand [trace annotations](/observability/trace-with-python-sdk/annotate-traces) for collecting feedback
- Check out the [Agenta UI guide](/observability/using-the-ui/filtering-traces) for filtering and analyzing traces